In [ ]:
# ================================
# Análise de Estratégia de Releases
# Projeto: AnythingLLM
# Modelo: microsoft/Phi-3.5-mini-instruct
# ================================

!pip install -q transformers accelerate requests bitsandbytes
!pip install --upgrade transformers

import requests
import torch
import json
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ----------------
# 1. Coleta mínima de dados do GitHub
# ----------------
REPO = "Mintplex-Labs/anything-llm"

def coletar_dados():
    releases = requests.get(
        f"https://api.github.com/repos/{REPO}/releases?per_page=10"
    ).json()

    commits = requests.get(
        f"https://api.github.com/repos/{REPO}/commits?per_page=30"
    ).json()

    return releases, commits

releases, commits = coletar_dados()

# Salvar dados coletados
with open("dados_anythingllm.json", "w", encoding="utf-8") as f:
    json.dump(
        {
            "repositorio": REPO,
            "coletado_em": datetime.now().isoformat(),
            "releases": releases,
            "commits": commits
        },
        f,
        indent=2,
        ensure_ascii=False
    )

# ----------------
# 2. Preparação dos dados para o LLM
# ----------------
release_info = []
for r in releases:
    tag = r.get("tag_name", "N/A")
    date = r.get("published_at", "N/A")
    release_info.append(f"{tag} - {date}")

prompt = f"""
Você é um especialista em Engenharia de Software.

Com base APENAS nos dados abaixo, identifique a ESTRATÉGIA DE RELEASES
do projeto AnythingLLM.

DADOS DE RELEASES (tag - data):
{chr(10).join(release_info)}

TOTAL DE RELEASES ANALISADAS: {len(releases)}
TOTAL DE COMMITS RECENTES ANALISADOS: {len(commits)}

CLASSIFIQUE A ESTRATÉGIA COMO UMA DAS SEGUINTES:
- Rapid Releases
- Release Train
- LTS + Current
- Ad hoc / Irregular

RESPONDA NO FORMATO:
1. Estratégia identificada:
2. Frequência observada:
3. Uso de versionamento semântico (Sim/Não):
4. Justificativa baseada nos dados:
"""

# Salvar prompt
with open("prompt_anythingllm.json", "w", encoding="utf-8") as f:
    json.dump(
        {
            "modelo": "microsoft/Phi-3.5-mini-instruct",
            "prompt": prompt
        },
        f,
        indent=2,
        ensure_ascii=False
    )

# ----------------
# 3. Carregamento do modelo (Colab Free)
# ----------------
model_name = "microsoft/Phi-3.5-mini-instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

# ----------------
# 4. Execução da análise (UMA inferência)
# ----------------
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.1,
        do_sample=False
    )

resposta = tokenizer.decode(output[0], skip_special_tokens=True)

# ----------------
# 5. Salvar resultado final
# ----------------
with open("resultado_analise_anythingllm.json", "w", encoding="utf-8") as f:
    json.dump(
        {
            "repositorio": REPO,
            "modelo": model_name,
            "data_analise": datetime.now().isoformat(),
            "resultado": resposta
        },
        f,
        indent=2,
        ensure_ascii=False
    )

print("\n===== RESULTADO FINAL =====\n")
print(resposta)
print("\n📁 Arquivos gerados:")
print("- dados_anythingllm.json")
print("- prompt_anythingllm.json")
print("- resultado_analise_anythingllm.json")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]